In [1]:
import pandas as pd
import geopandas as gpd
import sqlalchemy as sa
import pyodbc
import tkinter as tk
from sqlalchemy import create_engine
from api_config import geocode_postcode, fetch_isochrone, PostcodeNotFound
from geo_config import isochrone_to_gdf, teams_to_gdf, filter_teams_by_minutes,list_business_units, apply_team_filters

In [2]:
# Define custom colour palette

humannature = {'GC Dark Green': '#294238',
               'GC Light Green': '#b2d235',
               'GC Mid Green': '#50b748',
               'GC Orange': '#f57821',
               'GC Light Grey': '#e6ebe3'
               }

In [3]:
# SQL server details 
server = 'dblistener1'
database = 'information_centre'
driver = 'ODBC Driver 17 for SQL Server' # Create SQLAlchemy engine and install necessary drivers
connection_string = f'mssql+pyodbc://{server}/{database}?driver={driver}'
engine = create_engine(connection_string)

# SQL query to select dataset from SSMS tables and save into allvisits dataframe
query = """
SELECT
    a.intContractorID,
    a.strName AS Contractor,
    e.BusinessUnitID,
    e.Name AS BusinessUnit,
    UPPER(a.strPostcode) AS Postcode,
    g.Latitude,
    g.Longitude,
    a.InternalContractor
FROM
    tblContractor a
    LEFT JOIN Contractor_Business_Unit d
        ON a.intContractorID = d.ContractorID
    LEFT JOIN Business_Unit e
        ON d.BusinessUnitID = e.BusinessUnitID
    LEFT JOIN Business_Unit_Master_Status f
        ON d.StatusID = f.StatusID
    LEFT JOIN dbs_PostCode.dbo.tblPostcodes_New g
        ON REPLACE(a.strPostcode, ' ', '') = g.PostcodeNoSpaces COLLATE Latin1_General_CI_AS
WHERE
    ISNULL(a.bDisabled, 0) = 0
    AND f.StatusID IN (60, 70, 80)
    AND ISNULL(a.IsTest, 0) = 0
    AND e.BusinessUnitID != 37;
"""
with engine.connect() as connection:
    fieldteams = pd.read_sql(query, connection)
    connection.close()

In [5]:
try:
    site_lon, site_lat = geocode_postcode("CM12 0EQ")
except PostcodeNotFound as e:
    # In the GUI: show a messagebox with str(e) and return early
    print(f"Postcode error: {e}")
else:
    # One API call to Mapbox Isochrone API
    iso_geojson = fetch_isochrone(site_lon, site_lat)

    # Filter locally using GeoPandas rather than multiple API calls
    iso_gdf   = isochrone_to_gdf(iso_geojson)
    teams_gdf = teams_to_gdf(fieldteams)

    # Filter teams within the 60-minute isochrone to reduce multiple API calls later
    filtered_teams = filter_teams_by_minutes(teams_gdf, iso_gdf, minutes=60)

# Business Unit options for dropdown
bu_options = list_business_units(filtered_teams)

# Example selections
chosen_bu = "Grounds Maintenance"         # or something from bu_options, e.g., "Arboriculture"
chosen_internal_flag = None # 1 = Directly Employed, 0 = Contractor, None = both

# Apply filters without making any OSRM calls yet
teams_ready_for_routing = apply_team_filters(
    filtered_teams,
    business_unit=chosen_bu,
    internal_flag=chosen_internal_flag
)


,intContractorID,Contractor,BusinessUnitID,BusinessUnit,Postcode,Latitude,Longitude,InternalContractor,geometry
0,4713,OPERATIONS London and East Region,7,Grounds Maintenance,EC3N 1LS,51.513279,-0.075934,1,POINT (-0.07593 51.51328)
1,7550,Manor Oaks Ltd,7,Grounds Maintenance,SS8 9QP,51.530073,0.573521,0,POINT (0.57352 51.53007)
2,7598,Knight Property Maintenance Ltd,7,Grounds Maintenance,CR08ED,51.377074,-0.026699,0,POINT (-0.0267 51.37707)
3,7126,Maple Tree Surgery and Landscapes Ltd,7,Grounds Maintenance,SS9 3JP,51.547693,0.663300,0,POINT (0.6633 51.54769)
4,6368,Lawn Enforcement Gardens and Trees,7,Grounds Maintenance,EN8 0LB,51.713218,-0.027866,0,POINT (-0.02787 51.71322)
5,5004,MJ Landscape,7,Grounds Maintenance,E13 9LL,51.527281,0.034240,0,POINT (0.03424 51.52728)
6,6780,SLP Vegetation Solutions,7,Grounds Maintenance,ME4 3HP,51.402797,0.543664,0,POINT (0.54366 51.4028)
7,8188,Oxleas Tree Care Ltd,7,Grounds Maintenance,BR76LH,51.413412,0.077958,0,POINT (0.07796 51.41341)
8,1363,G. B. Benningfield Ltd,7,Grounds Maintenance,CM3 6EQ,51.683220,0.773234,0,POINT (0.77323 51.68322)
9,5590,MM Services,7,Grounds Maintenance,CM3 6EQ,51.683220,0.773234,0,POINT (0.77323 51.68322)
